# Milestone project 2: ***SkimLit***

The purpose of this project is to build an NLP model to make reading medical abstracts easier.


In [2]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [3]:
## Get data
!git clone https://github.com/Franck-Dernoncourt/pubmed-rct.git

Cloning into 'pubmed-rct'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 39 (delta 8), reused 5 (delta 5), pack-reused 25 (from 1)
Receiving objects: 100% (39/39), 177.08 MiB | 45.47 MiB/s, done.
Resolving deltas: 100% (15/15), done.


In [4]:
!ls pubmed-rct

PubMed_200k_RCT
PubMed_200k_RCT_numbers_replaced_with_at_sign
PubMed_20k_RCT
PubMed_20k_RCT_numbers_replaced_with_at_sign
README.md


In [5]:
!ls pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign

dev.txt  test.txt  train.txt


In [6]:
# start our experiments using 20k datasets
data_dir = "pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/"

In [7]:
import os

filenames = [data_dir + filename for filename in os.listdir(data_dir)]
filenames

['pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/test.txt',
 'pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/dev.txt',
 'pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/train.txt']

In [8]:
def get_lines(filename):
    with open(filename,"r") as f:
        return f.readlines()

In [9]:
train_lines = get_lines(data_dir+"train.txt")
train_lines[:20]

['###24293578\n',
 'OBJECTIVE\tTo investigate the efficacy of @ weeks of daily low-dose oral prednisolone in improving pain , mobility , and systemic low-grade inflammation in the short term and whether the effect would be sustained at @ weeks in older adults with moderate to severe knee osteoarthritis ( OA ) .\n',
 'METHODS\tA total of @ patients with primary knee OA were randomized @:@ ; @ received @ mg/day of prednisolone and @ received placebo for @ weeks .\n',
 'METHODS\tOutcome measures included pain reduction and improvement in function scores and systemic inflammation markers .\n',
 'METHODS\tPain was assessed using the visual analog pain scale ( @-@ mm ) .\n',
 'METHODS\tSecondary outcome measures included the Western Ontario and McMaster Universities Osteoarthritis Index scores , patient global assessment ( PGA ) of the severity of knee OA , and @-min walk distance ( @MWD ) .\n',
 'METHODS\tSerum levels of interleukin @ ( IL-@ ) , IL-@ , tumor necrosis factor ( TNF ) - , and 

In [10]:
train_lines[0],train_lines[1],train_lines[110]

('###24293578\n',
 'OBJECTIVE\tTo investigate the efficacy of @ weeks of daily low-dose oral prednisolone in improving pain , mobility , and systemic low-grade inflammation in the short term and whether the effect would be sustained at @ weeks in older adults with moderate to severe knee osteoarthritis ( OA ) .\n',
 'RESULTS\tNon-transfer patients had less @-day death/myocardial infarction -LSB- @ % vs. @ % ( tertiary ) ; adjusted odds ratio ( OR ) : @ ( @-@ @ ) , P = @ -RSB- ; transferred ( @ % ) and tertiary patients were similar -LSB- adjusted OR : @ ( @-@ @ ) , P = @ -RSB- .\n')

In [11]:
len(train_lines[0])

12

In [12]:
def preprocess_text_with_line_numbers(filename):
    """
    return a list of dictionaries of abstract line data.
    """
    input_lines = get_lines(filename)
    abstract_lines = ""
    abstract_samples = []
    for line in input_lines:
        if line.startswith("###"):
            abstract_id = line
            abstract_lines = ""
        elif line.isspace():
            abstract_line_split = abstract_lines.splitlines()
            for abstract_line_number,abstract_line in enumerate(abstract_line_split):
                line_data = {}
                target_text_split = abstract_line.split("\t")
                line_data["target"] = target_text_split[0]
                line_data["text"] = target_text_split[1]
                line_data["line_number"] = abstract_line_number
                line_data["total_lines"] = len(abstract_line_split) - 1
                abstract_samples.append(line_data)
        else:
            abstract_lines += line
    return abstract_samples
             
            
            
    
        
    

In [13]:
train_data = preprocess_text_with_line_numbers(data_dir + "train.txt")
val_data = preprocess_text_with_line_numbers(data_dir + "dev.txt")
test_data = preprocess_text_with_line_numbers(data_dir + "test.txt")
train_data[:15]

[{'target': 'OBJECTIVE',
  'text': 'To investigate the efficacy of @ weeks of daily low-dose oral prednisolone in improving pain , mobility , and systemic low-grade inflammation in the short term and whether the effect would be sustained at @ weeks in older adults with moderate to severe knee osteoarthritis ( OA ) .',
  'line_number': 0,
  'total_lines': 11},
 {'target': 'METHODS',
  'text': 'A total of @ patients with primary knee OA were randomized @:@ ; @ received @ mg/day of prednisolone and @ received placebo for @ weeks .',
  'line_number': 1,
  'total_lines': 11},
 {'target': 'METHODS',
  'text': 'Outcome measures included pain reduction and improvement in function scores and systemic inflammation markers .',
  'line_number': 2,
  'total_lines': 11},
 {'target': 'METHODS',
  'text': 'Pain was assessed using the visual analog pain scale ( @-@ mm ) .',
  'line_number': 3,
  'total_lines': 11},
 {'target': 'METHODS',
  'text': 'Secondary outcome measures included the Western Ontari

In [14]:
# turn data into DataFrame for more visualize
import pandas as pd
train_data_df = pd.DataFrame(train_data)
val_data_df = pd.DataFrame(val_data)
test_data_df = pd.DataFrame(test_data)
train_data_df.head(14)

,target,text,line_number,total_lines
0,OBJECTIVE,To investigate the efficacy of @ weeks of dail...,0,11
1,METHODS,A total of @ patients with primary knee OA wer...,1,11
2,METHODS,Outcome measures included pain reduction and i...,2,11
3,METHODS,Pain was assessed using the visual analog pain...,3,11
4,METHODS,Secondary outcome measures included the Wester...,4,11
5,METHODS,"Serum levels of interleukin @ ( IL-@ ) , IL-@ ...",5,11
6,RESULTS,There was a clinically relevant reduction in t...,6,11
7,RESULTS,The mean difference between treatment arms ( @...,7,11
8,RESULTS,"Further , there was a clinically relevant redu...",8,11
9,RESULTS,These differences remained significant at @ we...,9,11


In [ ]:
test_data_df.head(10)

In [ ]:
val_data_df.head(10)

In [ ]:
train_data_df

In [ ]:
train_data_df.target.value_counts()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(train_data_df.target.value_counts())

In [ ]:
target_values = train_data_df.target.value_counts()
plt.bar(target_values.index,target_values.values,color="lightgreen")

In [ ]:
train_data_df.total_lines.plot.hist()

In [ ]:
plt.hist(train_data_df.total_lines)

In [ ]:
max(train_data_df.total_lines),min(train_data_df.total_lines),len(train_data_df.total_lines)

In [ ]:
train_sentences = train_data_df["text"].tolist()
val_sentences = val_data_df["text"].tolist()
test_sentences = test_data_df["text"].tolist()
train_sentences[:2]

In [ ]:
val_sentences[:4]

In [ ]:
test_sentences[:5]

## Make numeric labels

In [ ]:
train_data_df["target"].to_numpy()

In [ ]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

one_hot_encoder = OneHotEncoder(sparse_output=False)  # use sparse=False for dense NumPy array

train_labels_one_hot = one_hot_encoder.fit_transform(train_data_df["target"].to_numpy().reshape(-1, 1))
val_labels_one_hot   = one_hot_encoder.transform(val_data_df["target"].to_numpy().reshape(-1, 1))
test_labels_one_hot  = one_hot_encoder.transform(test_data_df["target"].to_numpy().reshape(-1, 1))


In [ ]:
train_labels_one_hot

## Label encoder labels

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_data_df["target"].to_numpy())
val_labels_encoded = label_encoder.transform(val_data_df["target"].to_numpy())
test_labels_encoded = label_encoder.transform(test_data_df["target"].to_numpy())

In [ ]:
train_labels_encoded

In [ ]:
test_labels_encoded[:5]

In [ ]:
# Get class names and number of classes from label encoder instances
num_classes = len(label_encoder.classes_)
class_names = label_encoder.classes_
num_classes,class_names

In [ ]:
class_names[1]

## Model 1: baseline, naive bayes model

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [ ]:
model_0 = Pipeline([
    ("tf_idf",TfidfVectorizer()),
    ("clf",MultinomialNB()),
    
])

model_0.fit(train_sentences,train_labels_encoded)

In [ ]:
model_0.score(val_sentences,val_labels_encoded)

In [ ]:
baseline_preds = model_0.predict(val_sentences)

In [ ]:
baseline_preds

In [ ]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/refs/heads/main/extras/helper_functions.py

In [ ]:
from helper_functions import calculate_results

In [ ]:
baseline_results = calculate_results(y_true=val_labels_encoded,y_pred=baseline_preds)
baseline_results

In [ ]:
train_sentences[:10]

## Model 3: 1D Conv layer
how to vectorize and then embedd words?
tokenize using keras tokenization class,
embedded tokenized words in higher order embeding using keras embedding class
build the model, compile the model and fit the model and finally play around with model hyperparameters, parameter tuning, evaluating,tweaking,...

In [ ]:
# total number of sentences
total_sentences = len(train_sentences)

In [ ]:
# total number of tokens
total_token = [len(i.split()) for i in train_sentences]


In [ ]:
# sequence_length = tf.round(total_token/total_sentences)
# sequence_length

In [ ]:
# the distribution of looks like
import matplotlib.pyplot as plt
plt.hist(total_token,bins=20)

In [ ]:
import numpy as np
output_seq_len = int(np.percentile(total_token,95))
output_seq_len

In [ ]:
from tensorflow.keras import layers
max_tokens = 68000
output_sequence_len = 55
text_vectorizer = layers.TextVectorization(max_tokens=max_tokens,
                                          standardize="lower_and_strip_punctuation",
                                           split="whitespace",
                                          encoding="utf-8",
                                          output_mode="int",
                                          name="Tokenization_layer",
                                          output_sequence_length=output_seq_len)


In [ ]:
text_vectorizer.adapt(train_sentences+val_sentences+test_sentences)

In [ ]:
import random
target_sentence = random.choice(train_sentences)
print(f"Text:\n{target_sentence}")
print(f"\nLength of text:{len(target_sentence.split())}")
print(f"\nVectorized text:{text_vectorizer(target_sentence)}")

In [ ]:
text_vectorizer.get_config()

In [ ]:
rct_20k_text_vocab = text_vectorizer.get_vocabulary()
len(rct_20k_text_vocab)


In [ ]:
# Embedding layer
token_embeded = layers.Embedding(input_dim=len(rct_20k_text_vocab),
                            output_dim=128,
                             mask_zero=True
                            )


In [ ]:
print(f"Sentence before vectorization:\n{target_sentence}")
vectorized_sentence = text_vectorizer([target_sentence])
print(f"\nsentence after vectorized\n: {vectorized_sentence}")
print(f"shape of vectorized sentence: {vectorized_sentence.shape}")
embeded_sentence = token_embeded(vectorized_sentence)
print(type(vectorized_sentence))
print(f"\nvectorized sentence after embeded:{embeded_sentence}")

In [ ]:
embeded_sentence.shape

## Creating datasets

In [ ]:
len(train_sentences),len(train_labels_one_hot[0])
train_labels_one_hot[0][10:],train_labels_one_hot.shape
print(train_labels_one_hot[0])


In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_sentences,train_labels_one_hot))
val_dataset = tf.data.Dataset.from_tensor_slices((val_sentences,val_labels_one_hot))
test_dataset = tf.data.Dataset.from_tensor_slices((test_sentences,test_labels_one_hot))


In [ ]:
train_labels_one_hot.shape

In [ ]:
train_dataset = train_dataset.batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
train_data_df.target.value_counts()

In [ ]:
inputs = layers.Input(shape=(1,),dtype=tf.string)
x = embedding(inputs)
x = layers.Conv1D(filters=10,
                  kernel_size= 4)(x)
x = layers.Dense(5,activation="softmax",name="output_layer")(x)
model_2 = tf.keras.Model(inputs,outputs)